In [1]:
# Define BaseModel as an abstract type, all models will belong to this category
abstract type BaseModel end
abstract type BaseModelFit{T<:BaseModel} end

# When we fit a model, we get back a ModelFit object, that has the original model, as well as results, and should not change once fitted
struct ModelFit{T} <: BaseModelFit{T}
    model :: T
    fit_result
end
model(modelFit::ModelFit) = modelFit.model # Accessor function for the family of ModelFit types, instead of directly accessing the field. This way the accessor function is already informed by the type of the model, as it infers it from the type of ModelFit it is accessing, and ends up being faster than using modelFit.model arbitrarily?

# Define a generic predict for BaseModelFit, that disambiguates them based on what Model they are the result of
predict(modelFit::BaseModelFit, Xnew) = predict(model(modelFit), modelFit, Xnew)


predict (generic function with 1 method)

In [2]:
```
Every model has 
- a unique name (that is the model type)
- a fit function (that dispatches based on first input of model type and returns a ModelFit type) and a 
- predict function (that dispatches based on the first input of model type, and second input of ModelFit type)
```
mutable struct LinModel <: BaseModel
    parameters # Maybe a dictionary of names and values for now
end

fit(model::LinModel, X::AbstractArray, y::AbstractArray) = ModelFit(model, model.parameters["x1"])
predict(model::LinModel, modelFit::BaseModelFit, Xnew) = 11

mutable struct NonLinModel <: BaseModel
    parameters # Maybe a dictionary of names and values for now
end

fit(model::NonLinModel, X::AbstractArray, y::AbstractArray) = ModelFit(model, 2)
predict(model::NonLinModel, modelFit::BaseModelFit, Xnew) = 22

predict (generic function with 3 methods)

In [3]:
lm = LinModel(Dict("x0" => 0, "x1" => 1))

LinModel(Dict("x1"=>1,"x0"=>0))

In [4]:
lm_fit = fit(lm, [],[])

ModelFit{LinModel}(LinModel(Dict("x1"=>1,"x0"=>0)), 1)

In [5]:
nlm_fit = ModelFit(NonLinModel("haha"),2)

ModelFit{NonLinModel}(NonLinModel("haha"), 2)

In [6]:
predict(lm_fit,[])

11

In [7]:
predict(nlm_fit,[])

22

### Tuning as a composable wrapper

In [8]:
# Define some types of tuning
abstract type BaseTuning end
struct SimpleGridTuning <: BaseTuning
    grid
end
struct ModelSelectTuning <: BaseTuning
    models
end

In [9]:
# Now imagine we want to add tuning to the model, this should result in a composite that inherits the initial model, as well as adds a tuning grid to change the parameters on
struct TunedModel{T<:BaseModel} <: BaseModel
    model :: T
    tuning :: BaseTuning
end

# Accessor functions (for compile-time lookup gain)
model(tunedModel::TunedModel) = tunedModel.model
tuning(tunedModel::TunedModel) = tunedModel.tuning

tuning (generic function with 1 method)

In [10]:
struct TunedModelFit{T} <: BaseModelFit{T}
    model :: T
    fit_result
    tuning :: BaseTuning
    tuning_result
end
model(modelFit::TunedModelFit) = modelFit.model

model (generic function with 3 methods)

In [11]:
# This is just an example of SimpleGridTuning, do_tuning dispatches on the type of tuning as well
function do_tuning(model::BaseModel, tuning::SimpleGridTuning, X, y)
    tuning_result = [fit(typeof(model)(parameters), X, y) for parameters in tuning.grid]
    best_result_ind = 2
    fit_result = tuning_result[best_result_ind] # Choose the best one given metric
    return TunedModelFit(
        typeof(model)(tuning.grid[best_result_ind]), 
        fit_result, 
        tuning, 
        tuning_result)
end

# This is just an example of ModelSelectTuning, do_tuning dispatches on the type of tuning as well
function do_tuning(model::BaseModel, tuning::ModelSelectTuning, X, y)
    tuning_result = [fit(cur_model, X, y) for cur_model in tuning.models]
    best_result_ind = 3
    fit_result = tuning_result[best_result_ind] # Choose the best one given metric
    return TunedModelFit(
        fit_result.model, 
        fit_result, 
        tuning, 
        tuning_result)
end


fit(tunedModel::TunedModel, X, y) = do_tuning(model(tunedModel), tuning(tunedModel), X, y)


fit (generic function with 3 methods)

In [12]:
curTuneGrid = [
        Dict("x0" => 0, "x1" => 1),
        Dict("x0" => 0, "x1" => 2),
        Dict("x0" => 0, "x1" => 3)
        ]

3-element Array{Dict{String,Int64},1}:
 Dict("x1"=>1,"x0"=>0)
 Dict("x1"=>2,"x0"=>0)
 Dict("x1"=>3,"x0"=>0)

In [13]:
lm_tuned = TunedModel(lm, SimpleGridTuning(curTuneGrid))

TunedModel{LinModel}(LinModel(Dict("x1"=>1,"x0"=>0)), SimpleGridTuning(Dict{String,Int64}[Dict("x1"=>1,"x0"=>0), Dict("x1"=>2,"x0"=>0), Dict("x1"=>3,"x0"=>0)]))

In [14]:
tuning(lm_tuned).grid

3-element Array{Dict{String,Int64},1}:
 Dict("x1"=>1,"x0"=>0)
 Dict("x1"=>2,"x0"=>0)
 Dict("x1"=>3,"x0"=>0)

In [15]:
typeof(lm_tuned)

TunedModel{LinModel}

In [16]:
lm_tuned_fit = fit(lm_tuned, [], [])

TunedModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), ModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), 2), SimpleGridTuning(Dict{String,Int64}[Dict("x1"=>1,"x0"=>0), Dict("x1"=>2,"x0"=>0), Dict("x1"=>3,"x0"=>0)]), ModelFit{LinModel}[ModelFit{LinModel}(LinModel(Dict("x1"=>1,"x0"=>0)), 1), ModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), 2), ModelFit{LinModel}(LinModel(Dict("x1"=>3,"x0"=>0)), 3)])

In [17]:
predict(lm_tuned_fit, [])

11

In [18]:
lm_tuned_fit.tuning_result

3-element Array{ModelFit{LinModel},1}:
 ModelFit{LinModel}(LinModel(Dict("x1"=>1,"x0"=>0)), 1)
 ModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), 2)
 ModelFit{LinModel}(LinModel(Dict("x1"=>3,"x0"=>0)), 3)

In [19]:
model(lm_tuned_fit)

LinModel(Dict("x1"=>2,"x0"=>0))

In [20]:
# Compose model selection and grid tuning
curTuneModels = [lm, NonLinModel("haha"), lm_tuned]
selectionTunedModel = TunedModel(lm, ModelSelectTuning(curTuneModels))

selectionTunedModel_fit = fit(selectionTunedModel, [], [])


TunedModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), TunedModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), ModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), 2), SimpleGridTuning(Dict{String,Int64}[Dict("x1"=>1,"x0"=>0), Dict("x1"=>2,"x0"=>0), Dict("x1"=>3,"x0"=>0)]), ModelFit{LinModel}[ModelFit{LinModel}(LinModel(Dict("x1"=>1,"x0"=>0)), 1), ModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), 2), ModelFit{LinModel}(LinModel(Dict("x1"=>3,"x0"=>0)), 3)]), ModelSelectTuning(BaseModel[LinModel(Dict("x1"=>1,"x0"=>0)), NonLinModel("haha"), TunedModel{LinModel}(LinModel(Dict("x1"=>1,"x0"=>0)), SimpleGridTuning(Dict{String,Int64}[Dict("x1"=>1,"x0"=>0), Dict("x1"=>2,"x0"=>0), Dict("x1"=>3,"x0"=>0)]))]), Any[ModelFit{LinModel}(LinModel(Dict("x1"=>1,"x0"=>0)), 1), ModelFit{NonLinModel}(NonLinModel("haha"), 2), TunedModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), ModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), 2), SimpleGridTuning(Dict{String,Int64}[Dict("x1"=>1,"x0"=>0), Dict("x1"

In [27]:
typeof(selectionTunedModel)

TunedModel{LinModel}

In [21]:
# The underlying tuned model still gets tuned by its own underlying fit function as the result (same result as calling fit on the tuned model directly, see lm_tuned_fit )
selectionTunedModel_fit.tuning_result[3]

TunedModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), ModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), 2), SimpleGridTuning(Dict{String,Int64}[Dict("x1"=>1,"x0"=>0), Dict("x1"=>2,"x0"=>0), Dict("x1"=>3,"x0"=>0)]), ModelFit{LinModel}[ModelFit{LinModel}(LinModel(Dict("x1"=>1,"x0"=>0)), 1), ModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), 2), ModelFit{LinModel}(LinModel(Dict("x1"=>3,"x0"=>0)), 3)])

In [22]:
lm_tuned_fit

TunedModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), ModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), 2), SimpleGridTuning(Dict{String,Int64}[Dict("x1"=>1,"x0"=>0), Dict("x1"=>2,"x0"=>0), Dict("x1"=>3,"x0"=>0)]), ModelFit{LinModel}[ModelFit{LinModel}(LinModel(Dict("x1"=>1,"x0"=>0)), 1), ModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), 2), ModelFit{LinModel}(LinModel(Dict("x1"=>3,"x0"=>0)), 3)])

#### Side design question: Should the returned model after ModelSelectionTuning be as simplified as possible, or retain its full complexity?

In [23]:
# At the moment it returns fitted version rather than the full underlying model (it does do the fitting to evaluate performnace properly, and that is stored in tuning_results as expected)
selectionTunedModel_fit.model

LinModel(Dict("x1"=>2,"x0"=>0))

In [24]:
selectionTunedModel_fit.tuning.models[3] # This returns the full underlying model that was further tuned

TunedModel{LinModel}(LinModel(Dict("x1"=>1,"x0"=>0)), SimpleGridTuning(Dict{String,Int64}[Dict("x1"=>1,"x0"=>0), Dict("x1"=>2,"x0"=>0), Dict("x1"=>3,"x0"=>0)]))

In [25]:
selectionTunedModel_fit.tuning_result[3] # With the results of tuning that model being here

TunedModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), ModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), 2), SimpleGridTuning(Dict{String,Int64}[Dict("x1"=>1,"x0"=>0), Dict("x1"=>2,"x0"=>0), Dict("x1"=>3,"x0"=>0)]), ModelFit{LinModel}[ModelFit{LinModel}(LinModel(Dict("x1"=>1,"x0"=>0)), 1), ModelFit{LinModel}(LinModel(Dict("x1"=>2,"x0"=>0)), 2), ModelFit{LinModel}(LinModel(Dict("x1"=>3,"x0"=>0)), 3)])

In [26]:
predict(selectionTunedModel_fit, []) # This just predicts based on the simplest underlying model that was selected after ModelSelectionTuning and SimpleGridTuning

11

In [ ]:
# Traits - old


# using SimpleTraits
# @traitdef IsLinearModelFit{X}
# @traitimpl IsLinearModelFit{X} <- (isa(X.model,LinModel)) # This is not the way to do this

# @traitfn predict{X; IsLinearModelFit{X}}(modelFit::X, Xnew) = 11
# @traitfn predict{X; !IsLinearModelFit{X}}(modelFit::X, Xnew) = 12